In [1]:
#Gen-2(time, iteration, csv file)
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import LinearNDInterpolator, griddata
import pandas as pd
import time
import re
import csv

d_s = 0.4

def critical_diameter(file_path, lower_radius, d_s):
    # Extract radius from the file path 
    radius_match = re.search(r"test_([\d]+\.[\d]+)_([\d]+)", file_path)  
    if radius_match:
        radius = float(radius_match.group(1)) * d_s/2
        n = int(radius_match.group(2))
    else:
        raise ValueError("Could not determine radius from file name.")
    
    tilt = d_s / n
    centers = [(0, 0), (0, d_s), (d_s, tilt), (d_s, d_s + tilt)]
    
    def compute_result(radius_of_particle, d_s):
        y_s = radius + radius_of_particle
        wall_distance_threshold = radius_of_particle

        # Start timer for simulation
        start_time = time.time()
        
        x_range = np.linspace(0, d_s, 400)
        y_range = np.linspace(0, d_s + tilt, 400)
        X, Y = np.meshgrid(x_range, y_range)
        
        distances = np.full_like(X, np.inf, dtype=float)
        for cx, cy in centers:
            distance_to_post = np.sqrt((X - cx) ** 2 + (Y - cy) ** 2) - radius
            distances = np.minimum(distances, distance_to_post)
        distances = np.maximum(distances, 0)
        
        dy, dx = np.gradient(distances, y_range, x_range)
        gradient_magnitude = np.sqrt(dx ** 2 + dy ** 2)
        
        normal_x = np.zeros_like(dx)
        normal_y = np.zeros_like(dy)
        non_zero_magnitude = gradient_magnitude > 0
        normal_x[non_zero_magnitude] = dx[non_zero_magnitude] / gradient_magnitude[non_zero_magnitude]
        normal_y[non_zero_magnitude] = dy[non_zero_magnitude] / gradient_magnitude[non_zero_magnitude]
        
        wall_distance_interp = LinearNDInterpolator(list(zip(X.ravel(), Y.ravel())), distances.ravel(), fill_value=np.inf)
        normal_x_interp = LinearNDInterpolator(list(zip(X.ravel(), Y.ravel())), normal_x.ravel(), fill_value=0)
        normal_y_interp = LinearNDInterpolator(list(zip(X.ravel(), Y.ravel())), normal_y.ravel(), fill_value=0)
        
        velocity_field_data = pd.read_csv(file_path)
        x = velocity_field_data['x'].values
        y = velocity_field_data['y'].values
        u = velocity_field_data['u'].values
        v = velocity_field_data['v'].values
        
        max_vel = max(np.max(np.abs(u)), np.max(np.abs(v)))
        u_interp = LinearNDInterpolator(list(zip(x, y)), u / max_vel)
        v_interp = LinearNDInterpolator(list(zip(x, y)), v / max_vel)
        
        def velocity(x, y):
            return u_interp(x, y), v_interp(x, y)
        
        def rk4_step(x, y, dt, u, v):
            k1x = dt * u
            k1y = dt * v
            u2, v2 = velocity(x + k1x / 2, y + k1y / 2)
            k2x = dt * u2
            k2y = dt * v2
            u3, v3 = velocity(x + k2x / 2, y + k2y / 2)
            k3x = dt * u3
            k3y = dt * v3
            u4, v4 = velocity(x + k3x, y + k3y)
            k4x = dt * u4
            k4y = dt * v4
            return x + (k1x + 2 * k2x + 2 * k3x + k4x) / 6, y + (k1y + 2 * k2y + 2 * k3y + k4y) / 6
        
        x0, y0 = 0, y_s
        dt = 0.0001
        x_traj, y_traj = [x0], [y0]
        u, v = velocity(x0, y0)
        
        max_iterations = 300000  # Set a maximum number of iterations
        iteration_count = 0
        
        while x_traj[-1] < 0.39999 and y_traj[-1] > 0.0001 and iteration_count < max_iterations:
            x_next, y_next = rk4_step(x_traj[-1], y_traj[-1], dt, u, v)
            distance_to_wall = wall_distance_interp(x_next, y_next)
            if distance_to_wall <= wall_distance_threshold:
                nx, ny = normal_x_interp(x_next, y_next), normal_y_interp(x_next, y_next)
                dot_product = u * nx + v * ny
                u = u - 2 * dot_product * nx
                v = v - 2 * dot_product * ny
                x_next += (wall_distance_threshold - distance_to_wall) * nx
                y_next += (wall_distance_threshold - distance_to_wall) * ny
            else:
                u, v = velocity(x_next, y_next)
            x_traj.append(x_next)
            y_traj.append(y_next)
            iteration_count += 1


        tm = time.time() - start_time

        return (1 if y_traj[-3] > y_s else -1), iteration_count, tm

    upper_radius = (d_s/2 - radius) * 0.9
    # Binary search method to find critical radius
    tolerance = 0.0001
    while abs(upper_radius - lower_radius) > tolerance:
        mid_radius = (upper_radius + lower_radius) / 2
        mid_result, iteration_count, tm = compute_result(mid_radius, d_s)
        print(f"Iteration count for radius {mid_radius}: {iteration_count} in {tm:.2f} seconds")
        if mid_result == -1:
            lower_radius = mid_radius
        else:
            upper_radius = mid_radius
    
    print(f"Critical radius range for {float(radius_match.group(1)):.2f} and {n}: Lower = {lower_radius:.8f} m, Upper = {upper_radius:.8f} m")

    return lower_radius, upper_radius, iteration_count, tm

In [2]:
#Gen-2, run code
file_paths = [
    rf'..\results\test\csv\test_{velocity}_{i}.csv'
    for velocity in [0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7]
    for i in [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
]
results = []
lower_radius = float(0.005)

for file_path in file_paths:
    try:
        lw, uw, iteration, tm = critical_diameter(file_path, lower_radius, d_s=0.4)
        velocity, i = re.search(r"test_([\d]+\.[\d]+)_(\d+)", file_path).groups()
        results.append([float(velocity), int(i), uw, lw])
    except ValueError as e:
        print(f"Error processing {file_path}: {e}")

# Convert results to numpy array for better handling
results_array = np.array(results)

# Save results to CSV with proper error handling
try:
    results_df = pd.DataFrame(
        results_array, 
        columns=['Velocity', 'i', 'Upper Width', 'Lower Width']
    )
    
    output_file = 'results/results_prediction.csv'
    results_df.to_csv(output_file, index=False)
    print(f"Results successfully saved to {output_file}")
    
    # Display summary statistics
    print("\nSummary Statistics:")
    print(results_df.describe())
    
except Exception as e:
    print(f"Error saving results: {e}")

Iteration count for radius 0.07000000000000002: 3585 in 8.81 seconds
Iteration count for radius 0.10250000000000004: 5524 in 14.43 seconds
Iteration count for radius 0.11875000000000004: 9026 in 14.28 seconds
Iteration count for radius 0.11062500000000003: 8767 in 14.29 seconds
Iteration count for radius 0.11468750000000003: 10835 in 14.40 seconds
Iteration count for radius 0.11265625000000003: 15281 in 14.89 seconds
Iteration count for radius 0.11164062500000002: 10318 in 14.31 seconds
Iteration count for radius 0.11214843750000003: 12184 in 14.51 seconds
Iteration count for radius 0.11240234375000002: 15628 in 14.75 seconds
Iteration count for radius 0.11252929687500002: 17199 in 15.04 seconds
Iteration count for radius 0.11246582031250002: 20718 in 15.23 seconds
Critical radius range for 0.25 and 3: Lower = 0.11240234 m, Upper = 0.11246582 m
Iteration count for radius 0.07000000000000002: 4567 in 8.59 seconds
Iteration count for radius 0.10250000000000004: 7842 in 14.16 seconds
Iter